In [4]:
!pip install tensorflow

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4835 sha256=081f43d6a495f93e578ef17892101727f43d9c747dd26201b1ad1249ce1cc2a9
  Stored in directory: c:\users\clockwork\appdata\local\pip\cache\wheels\3f\e3\ec\8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
Successfully built termcolor


In [5]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [23]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [7]:
train = pd.read_csv('train_data_labels.csv')
test = pd.read_csv('test_data.csv')

In [8]:
train.head()

,train_id,abstract,label
0,1,"save for some special cases, current training ...",cs
1,2,we consider a dynamical system with finitely m...,math.DS
2,3,"we consider discrete dynamical systems of ""ant...",cs
3,4,"retrofitting techniques, which inject external...",cs
4,5,approaches to decision-making under uncertaint...,cs


In [9]:
test.head()

,test_id,abstract
0,1,the method of model averaging has become an im...
1,2,unmanned aerial vehicle (uav) systems are bein...
2,3,"in this paper, we propose a new loss function ..."
3,4,we show how to integrate a weak morphism of li...
4,5,caustics occur widely in dynamics and take on ...


In [10]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.label.values)

In [12]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.abstract.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [13]:
print (xtrain.shape)
print (xvalid.shape)

(26674,)
(2964,)


## TF-IDF

In [14]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [18]:
print(xtrain_tfv)

  (0, 287418)	0.0557073019595801
  (0, 282839)	0.06146559195106117
  (0, 282819)	0.06338139570411896
  (0, 281718)	0.05091997291425598
  (0, 281282)	0.04641874291148448
  (0, 278056)	0.07059532361000515
  (0, 277662)	0.028890486415747724
  (0, 277500)	0.09683008985283903
  (0, 277091)	0.030316208024505176
  (0, 272795)	0.0950077022806325
  (0, 270549)	0.08727931476546415
  (0, 270535)	0.17355806602895235
  (0, 267910)	0.08584895620771142
  (0, 267619)	0.026546377376787077
  (0, 258517)	0.08653857159812164
  (0, 258511)	0.06639869188007293
  (0, 257675)	0.046522458068708604
  (0, 256659)	0.08248576189764963
  (0, 244868)	0.047948807618648
  (0, 244221)	0.08459789319449096
  (0, 244217)	0.0990605119811045
  (0, 244056)	0.0990605119811045
  (0, 244043)	0.07997279008836927
  (0, 240372)	0.0950077022806325
  (0, 240068)	0.03598998508299201
  :	:
  (26673, 88351)	0.26784354433457036
  (26673, 88338)	0.12872272311093155
  (26673, 83233)	0.13660464194069621
  (26673, 81717)	0.11609482937393661

In [20]:
# Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [21]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

In [24]:
print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 1.605 


In [26]:
# # Fitting a simple xgboost on tf-idf
# clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
#                         subsample=0.8, nthread=10, learning_rate=0.1)
# clf.fit(xtrain_tfv.tocsc(), ytrain)
# predictions = clf.predict_proba(xvalid_tfv.tocsc())

# print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

In [35]:
# # load the GloVe vectors in a dictionary:

# embeddings_index = {}
# f = open('glove.840B.300d.txt')
# for line in tqdm(f):
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

# print('Found %s word vectors.' % len(embeddings_index))